In [59]:
library(tidyverse)
library(RSQLite)
library(scales)
library(slider)
library(furrr)
library(xts)


tidy_finance <- dbConnect(
  SQLite(),
  "data/tidy_finance_r.sqlite",
  extended_types = TRUE
)

# List all tables in the database
dbListTables(tidy_finance)


# crsp_monthly <- tbl(tidy_finance, "crsp_monthly") |>
#   select(permno, month, industry, ret_excess) |>
#   collect()

# factors_ff3_monthly <- tbl(tidy_finance, "factors_ff3_monthly") |>
#   select(month, mkt_excess) |>
#   collect()

# crsp_monthly <- crsp_monthly |>
#   left_join(factors_ff3_monthly, join_by(month))

[1] "compustat"             "cpi_monthly"           "crsp_monthly"         
[4] "factors_ff3_daily"     "factors_ff3_monthly"   "factors_ff5_monthly"  
[7] "factors_q_monthly"     "industries_ff_monthly" "macro_predictors"

In [60]:
# Read tables from the database
crsp_monthly <- tbl(tidy_finance, "crsp_monthly") %>%
  collect()

compustat <- tbl(tidy_finance, "compustat") %>%
  collect()

cpi_monthly <- tbl(tidy_finance, "cpi_monthly") %>%
  collect()

factors_ff3_daily <- tbl(tidy_finance, "factors_ff3_daily") %>%
  collect()

factors_ff3_monthly <- tbl(tidy_finance, "factors_ff3_monthly") %>%
  collect()

factors_ff5_monthly <- tbl(tidy_finance, "factors_ff5_monthly") %>%
  collect()

factors_q_monthly <- tbl(tidy_finance, "factors_q_monthly") %>%
  collect()

industries_ff_monthly <- tbl(tidy_finance, "industries_ff_monthly") %>%
  collect()

macro_predictors <- tbl(tidy_finance, "macro_predictors") %>%
  collect()

library(dplyr)

sorted_sampled_crsp_wide <- crsp_monthly  %>%
  arrange(date)

print(head(sorted_sampled_crsp_wide))


# A tibble: 6 x 14
  permno date       month          ret   shrout altprc exchcd siccd mktcap
   <dbl> <date>     <date>       <dbl>    <dbl>  <dbl>  <int> <int>  <dbl>
1  10016 2000-01-31 2000-01-01 -0.0993 17869000  17         3  3560  304. 
2  10019 2000-01-31 2000-01-01 -0.416   8226000   5.88      3  3610   48.3
3  10025 2000-01-31 2000-01-01  0.238   7460000  31.9       3  3070  238. 
4  10001 2000-01-31 2000-01-01 -0.0441  2450000   8.12      3  4920   19.9
5  10026 2000-01-31 2000-01-01 -0.0671  9017000  19.1       3  2050  172. 
6  10002 2000-01-31 2000-01-01 -0.0256  7729000  11.9       3  6020   91.8
# i 5 more variables: mktcap_lag <dbl>, exchange <chr>, industry <chr>,
#   ret_excess <dbl>, gvkey <chr>


In [61]:
compustat


gvkey,datadate,seq,ceq,at,lt,txditc,txdb,itcb,pstkrv,...,oancf,sale,cogs,xint,xsga,be,op,year,at_lag,inv
<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
001004,2000-05-31,339.515,339.515,740.998,401.483,56.020,56.020,0,0.000,...,10.051,1024.333,833.107,23.431,102.195,395.535,0.165851315,2000,NA,NA
001004,2001-05-31,340.212,340.212,701.854,361.642,55.063,55.063,0,0.000,...,46.093,874.255,713.811,21.887,96.077,395.275,0.107469483,2001,740.998,-0.052826054
001004,2002-05-31,310.235,310.235,710.199,399.964,30.601,30.601,0,0.000,...,-33.315,638.721,526.477,19.798,85.037,340.836,0.021737727,2002,701.854,0.011889937
001004,2003-05-31,294.988,294.988,686.621,391.633,22.601,22.601,0,0.000,...,34.733,606.337,496.747,19.539,78.845,317.589,0.035284597,2003,710.199,-0.033199146
001004,2004-05-31,301.684,301.684,709.292,407.608,17.628,17.628,0,0.000,...,14.572,651.958,523.302,18.819,81.165,319.312,0.089793055,2004,686.621,0.033018215
001004,2005-05-31,314.744,314.744,732.230,417.486,18.089,18.089,0,0.000,...,50.938,747.848,598.172,16.417,87.902,332.833,0.136275550,2005,709.292,0.032339290
001004,2006-05-31,422.717,422.717,978.819,556.102,25.357,25.357,0,0.000,...,-40.482,897.284,704.081,18.004,101.326,448.074,0.164867857,2006,732.230,0.336764405
001004,2007-05-31,494.243,494.243,1067.633,573.390,40.121,40.121,0,0.000,...,-21.239,1061.169,837.171,17.678,105.091,534.364,0.189438286,2007,978.819,0.090735877
001004,2008-05-31,585.255,585.255,1362.010,776.755,28.011,28.011,0,0.000,...,16.926,1384.919,1080.895,21.950,135.502,613.266,0.239002325,2008,1067.633,0.275728645


In [62]:
print(length(unique(crsp_monthly$permno)))
print(dim(crsp_monthly))

[1] 12659
[1] 1242635      14


In [63]:
# # Load necessary libraries
# library(readr)

# Read the CSV file
wrds_stock_data <- read_csv("./data/wrds_stock_data.csv")

# Check the unique permno
unique_permno <- unique(wrds_stock_data$PERMNO)

# Print the unique permno
print(length(unique_permno))


Rows: 3600 Columns: 6
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (1): date
dbl (5): PERMNO, PRC, VOL, RET, SHROUT

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 100


In [64]:
library(tidyverse)
permnos <- read_csv("./data/all_permnos.txt",
                    col_names = FALSE) %>% pull()
set.seed(7244)
mypermnos = permnos %>% base::sample(100)
write.table(mypermnos,'mypermnos.txt',
            row.names=FALSE,col.names=FALSE)

print(length(mypermnos))

Rows: 489 Columns: 1


-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (1): X1

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 100


In [65]:
# Filter the crsp_monthly dataset to include only data from Jan 2000 to Dec 2023
filtered_crsp_monthly <- crsp_monthly %>%
  filter(month >= as.Date("1999-12-31") & month <= as.Date("2023-12-31"))

print(dim(filtered_crsp_monthly))


sampled_crsp_monthly <- filtered_crsp_monthly %>%
  filter(permno %in% mypermnos) %>%select(-c('month',"ret_excess", "industry",'shrout','altprc','exchcd','siccd','mktcap','mktcap_lag','exchange','gvkey'))  # Remove the columns "mkt_excess" and "industry"


print(dim(sampled_crsp_monthly))

print(sampled_crsp_monthly)

print(length(unique(sampled_crsp_monthly$permno)))



[1] 1242635      14
[1] 28799     3
# A tibble: 28,799 x 3
   permno date           ret
    <dbl> <date>       <dbl>
 1  10032 2004-05-28  0.0223
 2  10032 2004-04-30 -0.192 
 3  10032 2004-03-31 -0.0700
 4  10032 2004-02-27 -0.117 
 5  10032 2004-01-30  0.262 
 6  10032 2003-12-31 -0.0429
 7  10032 2003-11-28  0.0340
 8  10032 2003-10-31  0.116 
 9  10032 2003-09-30 -0.0588
10  10032 2003-08-29  0.144 
# i 28,789 more rows
[1] 100


In [66]:

# Pivot the data to wide format
sampled_crsp_wide <- sampled_crsp_monthly %>%
  pivot_wider(
    names_from = permno,
    values_from = ret
  )

# Check the dimensions of the wide format data
print(dim(sampled_crsp_wide))

[1] 288 101


In [67]:
library(dplyr)

sorted_sampled_crsp_wide <- sampled_crsp_wide %>%
  arrange(date)

print(head(sorted_sampled_crsp_wide))


# A tibble: 6 x 101
  date       `10032`  `10138` `10501` `10696` `10866`  `10932`  `11018` `11144`
  <date>       <dbl>    <dbl>   <dbl>   <dbl>   <dbl>    <dbl>    <dbl>   <dbl>
1 2000-01-31  0.0568  0.0525  -0.265  -0.114  -0.265  -0.00732 -0.127    0.102 
2 2000-02-29  0.214  -0.153   -0.0737 -0.197   0.0120 -0.0914  -0.0621   0.0741
3 2000-03-31  0.180   0.203    0.253   0.365   0.152   0.0888   0.0432   0.213 
4 2000-04-28  0.150  -0.0348  -0.0242  0.235  -0.146  -0.0637   0.00355  0.127 
5 2000-05-31  0.0897  0.00164 -0.0621  0.0177  0.165   0.0789   0.0247  -0.113 
6 2000-06-30  0.353   0.116    0.0887 -0.0749  0.0974 -0.0379   0.0302   0.0317
# i 92 more variables: `11154` <dbl>, `11308` <dbl>, `11368` <dbl>,
#   `11369` <dbl>, `11397` <dbl>, `11481` <dbl>, `11775` <dbl>, `11752` <dbl>,
#   `11850` <dbl>, `12036` <dbl>, `13688` <dbl>, `13901` <dbl>, `14541` <dbl>,
#   `15069` <dbl>, `16548` <dbl>, `16678` <dbl>, `17743` <dbl>, `18075` <dbl>,
#   `18649` <dbl>, `20482` <dbl>, `

In [68]:
library(xts)

# Remove the "month" column from the dataframe
sampled_crsp_xts <- sampled_crsp_wide %>%
  select(-date)

# Convert the remaining columns to an xts object
sampled_crsp_xts <- xts(as.matrix(sampled_crsp_xts), order.by = sampled_crsp_wide$date)

# Rename the columns with mypermnos
colnames(sampled_crsp_xts) <- colnames(sampled_crsp_wide)[-1]  # Exclude the "month" column

# Print the first few rows of the xts object
print(head(sampled_crsp_xts))


              10032     10138     10501     10696     10866     10932     11018
2000-01-31 0.056818  0.052454 -0.265464 -0.114192 -0.265487 -0.007321 -0.126506
2000-02-29 0.214382 -0.152733 -0.073684 -0.197053  0.012048 -0.091398 -0.062069
2000-03-31 0.179856  0.203188  0.253030  0.364679  0.152381  0.088757  0.043235
2000-04-28 0.150094 -0.034810 -0.024242  0.235294 -0.145833 -0.063696  0.003546
2000-05-31 0.089723  0.001639 -0.062112  0.017687  0.164634  0.078947  0.024735
2000-06-30 0.353293  0.116334  0.088742 -0.074866  0.097382 -0.037940  0.030207
               11144     11154     11308     11368     11369     11397
2000-01-31  0.102041  0.045852 -0.013949  0.000000 -0.115183 -0.140845
2000-02-29  0.074074  0.421712 -0.153428  0.019608 -0.133136 -0.057377
2000-03-31  0.212759 -0.170338 -0.031208  0.076923  0.212833  0.121391
2000-04-28  0.126984 -0.056637  0.006658 -0.089286 -0.008523 -0.054688
2000-05-31 -0.112676 -0.279550  0.129630  0.129412 -0.128940 -0.016529
2000-06-30  0.

In [69]:
# # # Join with factors_ff3_monthly dataset
# # sampled_crsp_monthly <- sampled_crsp_monthly %>%
# #   left_join(factors_ff3_monthly, by = "month")
